## Compile raw outputs from diamond with count data

In [ ]:
library(plyr);library(dplyr);library(reshape2)

In [ ]:
# Generate raw count file for downstream analysis
# raw_count_data_07262018.RData # raw counts compiled

In [ ]:
# Run below example 'for loop' to compile output from diamond results

# For loop below reads in each XX_tax_assign.txt and XX_KO_assign.txt for each sample (list above). Then joined each one by the read ID, aggregates by the taxonomy and KO annotation information. Then writes this as a XX_compiled.txt file.
# july<-as.factor(c("July_1000m"))
# for (x in july){
#   tax_IN<-read.delim(paste(x,"tax_assign.txt",sep="_"), header = FALSE)
#   colnames(tax_IN)[1:2]<-c("Seq","Taxonomy")
#   KO_IN<-read.delim(paste(x,"KO_assign.txt",sep="_"), header=FALSE)
#   colnames(KO_IN)[1:2]<-c("Seq","KO")
#   tmp<-join(tax_IN, KO_IN, by="Seq", type="left", match="first")
#   # dim(compiled_df);dim(tax_IN);length(unique(tax_IN$Seq));dim(KO_IN);length(unique(KO_IN$Seq))
#   tmp$Sample<-x
#   tmp$Count<-1
#   tmp2<-aggregate(tmp$Count, by=list(Sample=tmp$Sample,Taxonomy=tmp$Taxonomy, KO=tmp$KO),sum)
#   colnames(tmp2)[4]<-"Count"
#   write.table(tmp2, file=(paste(x,"compiled.txt", sep="_")), row.names=FALSE, quote=FALSE, sep="\t")
#   rm(tmp); rm(tmp2)
# }

# Report taxonomy and KO assignment details:
# How many reads (what percent of reads) were given tax assignment, KO assignment, both tax and KO assignment, no assignment?
# How many reads were assigned to various taxonomic group levels?

In [ ]:
# Get tax and KO annotation information -----------------------------------
library(dplyr)
library(reshape2)
pre_list<-as.factor(c("Catalina_19","Catalina_20","Catalina_21","Catalina_22","Catalina_23","Catalina_24","July_1000m","July_150m","July_5m_Rep1","July_5m_Rep2","July_DCM_Rep1","July_DCM_Rep2","March_1000m","March_150m","March_5m_Rep1","March_5m_Rep2","March_DCM_Rep1","March_DCM_Rep2","Port_of_LA_1","Port_of_LA_2","Port_of_LA_3","Port_of_LA_4","Port_of_LA_5","Port_of_LA_6","SPOT_150m_25","SPOT_150m_26","SPOT_150m_27","SPOT_890m_28","SPOT_890m_29","SPOT_890m_30","SPOT_890m_31","SPOT_surface_10","SPOT_surface_11","SPOT_surface_12","SPOT_surface_13","SPOT_surface_14","SPOT_surface_15","SPOT_surface_16","SPOT_surface_17","SPOT_surface_18","SPOT_surface_7","SPOT_surface_8","SPOT_surface_9"))
# Generate blank slate data frames that will be populated with annotation information for each sample:
annotation_info<-data.frame(Sample = factor(),
                        TAX_KO_sum = double(),
                        TAX_only_sum = double(),
                        KO_only_sum = double(),
                        Total_wHits_sum = double(),
                        TAX_KO_perc_sum = double(),
                        TAX_only_perc_sum = double(),
                        KO_only_perc_sum = double(),
                        TAX_KO = double(),
                        TAX_only = double(),
                        KO_only = double(),
                        Total_wHits = double(),
                        TAX_KO_perc = double(),
                        TAX_only_perc = double(),
                        KO_only_perc = double())
tax_info<-data.frame(Sample = factor(),
                     to_Supergroup_sum=double(),
                     to_Phylum_sum=double(),
                     to_Class_sum=double(),
                     to_Order_sum=double(),
                     to_Family_sum=double(),
                     to_Genus_sum=double(),
                     to_Species_sum=double(),
                     to_Supergroup=double(),
                     to_Phylum=double(),
                     to_Class=double(),
                     to_Order=double(),
                     to_Family=double(),
                     to_Genus=double(),
                     to_Species=double())
annotation_info
tax_info
# Function below will read in each of the XX_compiled.txt files, extract information on reads with respect to their annotation (see above questions).
for (x in pre_list){
  file_in<-read.delim(paste(x,"compiled.txt",sep="_"), header = FALSE)
  colnames(file_in)[1:3]<-c("Taxonomy", "KO", "Count")
  file_in$Sample <-x
  file_in$occur<-1
  tmp_annotation<- file_in %>%
    group_by(Sample) %>%
    summarize(TAX_KO_sum=sum(Count[Taxonomy != "Not assigned" & KO != "Not assigned"]),
              TAX_only_sum=sum(Count[Taxonomy != "Not assigned" & KO == "Not assigned"]),
              KO_only_sum=sum(Count[Taxonomy == "Not assigned" & KO != "Not assigned"]),
              Total_wHits_sum=sum(TAX_KO_sum, TAX_only_sum, KO_only_sum),
              TAX_KO_perc_sum=paste(100*(TAX_KO_sum /Total_wHits_sum)),
              TAX_only_perc_sum=paste(100*(TAX_only_sum / Total_wHits_sum)),
              KO_only_perc_sum=paste(100*(KO_only_sum / Total_wHits_sum)),
              TAX_KO=sum(occur[Taxonomy != "Not assigned" & KO != "Not assigned"]),
              TAX_only=sum(occur[Taxonomy != "Not assigned" & KO == "Not assigned"]),
              KO_only=sum(occur[Taxonomy == "Not assigned" & KO != "Not assigned"]),
              Total_wHits=sum(TAX_KO, TAX_only, KO_only),
              TAX_KO_perc=paste(100*(TAX_KO /Total_wHits)),
              TAX_only_perc=paste(100*(TAX_only / Total_wHits)),
              KO_only_perc=paste(100*(KO_only / Total_wHits))) %>%
    as.data.frame
  annotation_info <- rbind(annotation_info, tmp_annotation)
  rm(tmp_annotation)
  # Now generate information about taxonomy assignment
  tax<-colsplit(file_in$Taxonomy, ";", c("Supergroup", "Phylum", "Class", "Order", "Family", "Genus", "Species"))
  df_wtax<-cbind(file_in, tax)
  df_wtax$Species<-as.character(gsub(";","",df_wtax$Species))
  df_wtax$occur<-1
  tmp_tax<-df_wtax %>%
    group_by(Sample) %>%
    summarize(Total_sum=sum(Count),
              to_Supergroup_sum=sum(Count[Supergroup != "Not assigned" & Phylum == ""& Class == ""& Order == ""& Family == "" & Genus == "" & Species == ""]), 
              to_Phylum_sum=sum(Count[Supergroup != "Not assigned" & Phylum != ""& Class == ""& Order == ""& Family == "" & Genus == "" & Species == ""]),
              to_Class_sum=sum(Count[Supergroup != "Not assigned" & Phylum != ""& Class != ""& Order == ""& Family == "" & Genus == "" & Species == ""]),
              to_Order_sum=sum(Count[Supergroup != "Not assigned" & Phylum != ""& Class != ""& Order != ""& Family == "" & Genus == "" & Species == ""]),
              to_Family_sum=sum(Count[Supergroup != "Not assigned" & Phylum != ""& Class != ""& Order != ""& Family != "" & Genus == "" & Species == ""]),
              to_Genus_sum=sum(Count[Supergroup != "Not assigned" & Phylum != ""& Class != ""& Order != ""& Family != "" & Genus != "" & Species == ""]),
              to_Species_sum=sum(Count[Supergroup != "Not assigned" & Phylum != ""& Class != ""& Order != ""& Family != "" & Genus != "" & Species != ""]),
              Total=sum(occur),
              to_Supergroup=sum(occur[Supergroup != "Not assigned" & Phylum == ""& Class == ""& Order == ""& Family == "" & Genus == "" & Species == ""]), 
              to_Phylum=sum(occur[Supergroup != "Not assigned" & Phylum != ""& Class == ""& Order == ""& Family == "" & Genus == "" & Species == ""]),
              to_Class=sum(occur[Supergroup != "Not assigned" & Phylum != ""& Class != ""& Order == ""& Family == "" & Genus == "" & Species == ""]),
              to_Order=sum(occur[Supergroup != "Not assigned" & Phylum != ""& Class != ""& Order != ""& Family == "" & Genus == "" & Species == ""]),
              to_Family=sum(occur[Supergroup != "Not assigned" & Phylum != ""& Class != ""& Order != ""& Family != "" & Genus == "" & Species == ""]),
              to_Genus=sum(occur[Supergroup != "Not assigned" & Phylum != ""& Class != ""& Order != ""& Family != "" & Genus != "" & Species == ""]),
              to_Species=sum(occur[Supergroup != "Not assigned" & Phylum != ""& Class != ""& Order != ""& Family != "" & Genus != "" & Species != ""])) %>%
    as.data.frame
  tax_info<-rbind(tax_info, tmp_tax)
  rm(tmp_tax)
}
head(annotation_info) # Sum of counts that have KO and/or tax IDs
head(tax_info) # Sum of counts that are assigned to taxonomic levels
write.table(annotation_info,file="Annotation_stats.txt", sep="\t", row.names=FALSE, quote=FALSE)
write.table(tax_info, "Tax_assignment_stats.txt", sep="\t", row.names=FALSE, quote=FALSE)

In [ ]:
# Compile counts from each sample
long_count_data<-data.frame(Sample = factor(),
                            Taxonomy = double(),
                            KO = double(),
                            Count = double())
#
#
for (x in pre_list){
  file_in<-read.delim(paste(x,"compiled.txt",sep="_"), header = FALSE)
  colnames(file_in)[1:3]<-c("Taxonomy", "KO", "Count")
  file_in$Sample <-x
  file_in$Count <-as.numeric(file_in$Count)
  long_count_data<-rbind(long_count_data, file_in)
  rm(file_in)
}
long_count_data<-rbind(long_count_data, new)
unique(long_count_data$Sample)
#
# Fix variable/sample names:
long_count_data$Sample<-(as.character(gsub("Catalina","Catalina_surface",long_count_data$Sample)))
long_count_data$Sample<-(as.character(gsub("Port_of_LA","PortofLA_surface",long_count_data$Sample)))
#
wide_count_data<-dcast(long_count_data, Taxonomy+KO~Sample,fun.aggregate = sum,value.var= "Count")
# head(long_count_data)
head(wide_count_data)
save(long_count_data, wide_count_data, file="raw_count_data_07262018.RData")
write.table(wide_count_data, file = "raw-count-data-metaT-ALOHA-SPOT.txt", row.names = FALSE, quote = FALSE, sep = "\t")

Use 'raw-count-data-metaT-ALOHA-SPOT.txt' and 'raw_count_data_07262018.RData' in all downstream analysis. 